In [110]:
import json
from dotenv import load_dotenv
from functools import reduce
import numpy as np
import operator
import os
import pandas as pd
import pymongo
from pymongo import MongoClient
import requests

In [14]:
load_dotenv()

True

#### SAO PAULO: Zona Central

In [27]:
sao_paulo_zc = {'type': 'Point', 'coordinates': [-23.533215960575276,-46.639408758375076]}

In [28]:
def getdata_fsq(url, city, venue):
    client_id = os.getenv("tok1")
    client_secret = os.getenv("tok2")
    url_ = f'{url}'
    c = city['coordinates'][0]
    c1 = city['coordinates'][1]
    parametros = {"client_id": client_id, "client_secret": client_secret, "v": "20180323", "ll": f'{c}, {c1}',
                  "query": venue, "limit": 55}
    resp = requests.get(url_, params=parametros).json()
    return resp

#### HACER UNA FUNCION PARA GENERAR DATAS POR CADA RUBRO, POR CADA CIUDAD

In [29]:
cathegories_s = ['vegan', 'airport', 'estacao', 'starbucks', 'veterinaria', 'basketball', 'escola']

In [30]:
def datagenerator(url, city, cathegories):
    cath = cathegories
    data_ = []
    for c in cath:
        data = getdata_fsq(url, city, c)
        data_.append(data['response']['venues'])
    return data_

In [31]:
sao_paulo = datagenerator('https://api.foursquare.com/v2/venues/search', sao_paulo_zc, cathegories_s)

In [40]:
def getFromDict(dic,map_):
    return reduce(operator.getitem, map_, dic)

def type_point(gps):
    return {"type": "Point", "coordinates": gps}

def jsonxport(name, l):
    n = f'{name}.json'
    with open(n, 'w') as f:
        json.dump(l, f)

#### OJO, ACA INVERTI EL ORDEN, LONG LAT

In [41]:
def jsongenerator(data, name):
    d = data
    map_name = ['name']
    map_categories = ['categories']
    map_lat = ['location', 'lat']
    map_long = ['location', 'lng']
    l = []
    for dic in d:
        ndic = {}
        ndic['name']= getFromDict(dic,map_name)
        ndic['categories']= getFromDict(dic,map_categories)
        ndic['latitude']= getFromDict(dic,map_lat)
        ndic['longitude']= getFromDict(dic,map_long)
        ndic['location']= type_point([ndic["longitude"], ndic["latitude"]])
        l.append(ndic)
    return jsonxport(name, l)

#### ACA FUNCION?

In [42]:
a = jsongenerator(sao_paulo[0], 'vegansp')

In [43]:
b = jsongenerator(sao_paulo[1], 'airportsp')

In [44]:
c = jsongenerator(sao_paulo[2], 'stationp')

In [45]:
d = jsongenerator(sao_paulo[3], 'starbucksp')

In [46]:
e = jsongenerator(sao_paulo[4], 'veterinarysp')

In [47]:
f = jsongenerator(sao_paulo[5], 'basketballsp')

In [48]:
g = jsongenerator(sao_paulo[6], 'schoolssp')

#### llamada a la collection

In [49]:
client = MongoClient("localhost:27017")
db = client.get_database("GeoSpatialProyect")

In [50]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoSpatialProyect')

In [80]:
g = db.get_collection("transport")

In [81]:
g.find_one().keys()

dict_keys(['_id', 'name', 'categories', 'latitude', 'longitude', 'location'])

In [85]:
cond1 = {'categories.name': 'Bus Line'}
cond2 = {"location": {"$near": {"$geometry": {'type': 'Point', 'coordinates': [-46.639408758375076,-23.533215960575276]}, "$minDistance": 0, "$maxDistance": 5000}}}

In [86]:
query = {'$and': [cond1, cond2]}

In [87]:
list(g.find(query))

[{'_id': ObjectId('6134d1263681de6a29cda132'),
  'name': 'Airport Bus Service',
  'categories': [{'id': '4bf58dd8d48988d12b951735',
    'name': 'Bus Line',
    'pluralName': 'Bus Lines',
    'shortName': 'Bus',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/busstation_',
     'suffix': '.png'},
    'primary': True}],
  'latitude': -23.543085,
  'longitude': -46.643772,
  'location': {'type': 'Point', 'coordinates': [-46.643772, -23.543085]}},
 {'_id': ObjectId('6134d1263681de6a29cda134'),
  'name': 'Airport Bus Service',
  'categories': [{'id': '4bf58dd8d48988d12b951735',
    'name': 'Bus Line',
    'pluralName': 'Bus Lines',
    'shortName': 'Bus',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/busstation_',
     'suffix': '.png'},
    'primary': True}],
  'latitude': -23.552027262857315,
  'longitude': -46.6481955585754,
  'location': {'type': 'Point',
   'coordinates': [-46.6481955585754, -23.552027262857315]}},
 {'_id': ObjectId('6134d1

In [129]:
score_p = {'Conditions': ['Airports', 'Commuter Station', 'Airport Shuttle', 'Schools', 'Vegan Restaurants', 'Veterinaries', 'Starbucks Cafe', 'Basketball Court'],
           'Geoqueries': [2.0, 3.0, 4.0, 8.0, 4.0, 2.0, 1.0, 3.], 'Weights': [0.19, 0.18, 0.17, 0.14, 0.12, 0.1, 0.05, 0.05]}

In [130]:
score_p = pd.DataFrame(score_p)

In [131]:
score_p

,Conditions,Geoqueries,Weights
0,Airports,2.0,0.19
1,Commuter Station,3.0,0.18
2,Airport Shuttle,4.0,0.17
3,Schools,8.0,0.14
4,Vegan Restaurants,4.0,0.12
5,Veterinaries,2.0,0.10
6,Starbucks Cafe,1.0,0.05
7,Basketball Court,3.0,0.05


In [132]:
w = np.array([0.19, 0.18, 0.17, 0.14, 0.12, 0.1, 0.05, 0.05])

In [133]:
g = np.array([2, 3, 4, 8, 4, 2, 1, 3])

In [134]:
s = w * g

In [135]:
score_p['Score'] = s

In [136]:
score_p

,Conditions,Geoqueries,Weights,Score
0,Airports,2.0,0.19,0.38
1,Commuter Station,3.0,0.18,0.54
2,Airport Shuttle,4.0,0.17,0.68
3,Schools,8.0,0.14,1.12
4,Vegan Restaurants,4.0,0.12,0.48
5,Veterinaries,2.0,0.10,0.20
6,Starbucks Cafe,1.0,0.05,0.05
7,Basketball Court,3.0,0.05,0.15
